Funzione di importazione file probabilità e dataset

In [1]:
import pickle
import pandas as pd
import numpy as np

#deserialize data from a file
def load_data(file_name,language):
    path="../data/"+language+"/"+file_name
    try: 
        file = open(path, 'rb') 
        data = pickle.load(file) 
        return data
    except: 
        print("Error in reading data")


# function to read data from file
def read_dataset(file_name,language):
    path="../data/"+language+"/dataset/"+file_name+".conllu"
    data = pd.read_csv (path, sep = '\t',quoting=3, names=["POSITION","WORD","TAG"])
    return data





Fuzione di salvataggio tagging in formato conllu

In [2]:
from conllu import parse, TokenList

# Funzione per salvare il DataFrame in un file CoNLL-U
def save_to_conllu(dataframe,file_name,language):
    # Creazione della lista di token da DataFrame
    path="../data/"+language+"/tagging/"+file_name
    tokens = []
    for _, row in dataframe.iterrows():
        token = {
            "id": row['POSITION'],
            "form": row['WORD'],
            "misc":  row['TAG']
        }
        tokens.append(token)
    
    # Creazione dell'oggetto TokenList
    token_list = TokenList(tokens)
    
    # Scrittura del TokenList nel file CoNLL-U
    with open(path, "w", encoding="utf-8") as f:
        f.write(token_list.serialize())
    print("DataFrame salvato in formato CoNLL-U:", path)



Funzioni di manipolazione e creazione del golden system

In [3]:
def extract_sentences_from_dataframe(df):
    sentences = ''
    for index, row in df.iterrows():
        word = row['WORD']
        if pd.notnull(word):  # Se la parola non è nulla
            if sentences:  # Se c'è già una frase, aggiungi uno spazio prima della nuova parola
                sentences += ' '
            sentences += word
    return sentences



def create_golden_dataframe(sentence, golden_tag):
    # Dividi la frase e i tag dorati in parole e tag
    words = sentence.split()
    tags = golden_tag.split(',')
    
    # Controlla se il numero di parole e tag è lo stesso
    if len(words) != len(tags):
        raise ValueError("Il numero di parole e tag dorati non corrisponde.")

    # Creazione del DataFrame dorato
    df = pd.DataFrame({'WORD': words, 'TAG': tags})
    df['POSITION'] = df.index + 1  # Aggiunge la colonna POSITION
    df = df[['POSITION', 'WORD', 'TAG']]  # Riordina le colonne
    
    return df

# Esempio di utilizzo
#sentence = "In 1975 , the Princess was listed among women with whom actor Warren Beatty had had romantic relationships ."
#golden_tag = "O,O,O,O,O,O,O,O,O,O,O,O,B-PER,I-PER,O,O,O,O,O"
#print(golden_df)


Viterbi

In [4]:
# Funzione per l'algoritmo di Viterbi: prende in input le probabilità di emissione e di transizione sottoforma di dataframe
# e restituisce le coppie parola-NER_TAG assegnate
import pandas as pd

def Viterbi(emission_dataframe, transition_dataframe):
    epsilon = 1e-10  # Valore molto piccolo
    # Lista delle parole nel testo
    testo_array = emission_dataframe.columns.tolist()
    # Numero di stati (tag)
    K = transition_dataframe.shape[0]
    
    # Lunghezza del testo
    T = len(testo_array)
    
    # Inizializzazione
    viterbi = np.zeros((K, T))
    backpointer = np.zeros((K, T), dtype=int)
    
    # Inizializzazione del primo passo
    Pi = 1  # Start equiprobabile 
    viterbi[:, 0] = np.log(emission_dataframe.iloc[:, 0] + epsilon) + np.log(Pi)
    backpointer[:, 0] = 0 
    
    # Iterazione
    for t in range(1, T):
        for k in range(K):
            # Calcolo della probabilità Viterbi per ogni stato
            max_val = float('-inf')
            max_idx = 0
            for j in range(K):
                val = viterbi[j, t - 1] + np.log(transition_dataframe.iloc[j, k] + epsilon) + np.log(emission_dataframe.iloc[k, t] + epsilon)
                if val > max_val:
                    max_val = val
                    max_idx = j
            viterbi[k, t] = max_val
            backpointer[k, t] = max_idx
    
    # Terminazione: Trova il massimo della colonna finale
    max_final_val = np.max(viterbi[:, T - 1])
    max_final_idx = np.argmax(viterbi[:, T - 1])
    
    # Costruzione del percorso Viterbi partendo dall'ultimo passo
    viterbi_path = [max_final_idx]
    for i in reversed(range(1, T)):
        viterbi_path.insert(0, backpointer[viterbi_path[0], i])
    
    # Costruzione delle coppie parola-tag assegnate
    word_tag_pairs = [(testo_array[i], transition_dataframe.index[j]) for i, j in enumerate(viterbi_path)]
    
    # Stampa del percorso Viterbi e delle coppie parola-tag assegnate
    print("Percorso Viterbi:", viterbi_path)

    df = pd.DataFrame({
        'POSITION': range(1, T + 1),
        'WORD': [pair[0] for pair in word_tag_pairs],
        'TAG': [pair[1] for pair in word_tag_pairs]
    })
    
    return df
    

Creazione del sub-dataset di probabilità di emissione per le parole di una frase.

Applicazione di diverse tecniche di smoothing per gestire le parole sconosciute:

1 - Sempre O: P(unk|O) = 1

2 - Sempre O o MISC: P(unk|O)=P(unk|B-MISC)=0.5

3 - Uniforme: P(unk|tag) = 1/#(NER_TAGs)

4 - Statistica TAG sul val set: parole che compaiono 1 sola volta  -> unknown_prob calcolata nel file learning 


In [5]:
#prende in input una frase, le probabilità di emisione e transizione apprese 
#restituisce le coppie parola-NER_TAG assegnate utilizzano l'algoritmo di Viterbi e applicando la tecnica di smoothing specificata
def viterbi_tagger(sentence, emission_prob, transition_prob, unkown_prob, smoothing_type=1):
    #inizializzazione
    tags=transition_prob.keys()
    words = sentence.split()
    transition_df = pd.DataFrame.from_dict(transition_prob)
    emission_sentence_df = pd.DataFrame(columns=words,index=tags)
    
    #iterazione per ogni parola delle frase aggiorna il dataframe delle emissioni
    for word in words:
        if word in emission_prob:
            #
            # Da Controllare Qui, sembra dia delle percentuali errate
            #
            emission_sentence_df[word] =  pd.Series(emission_prob[word]).iloc[:]

        else: #applicazione dello smoothing
           if (smoothing_type==1): emission_sentence_df[word] =  {tag: 0.99 if tag == "O" else 0.01 for tag in tags}
           elif (smoothing_type==2): emission_sentence_df[word] =  {tag: 0.5 if tag == "B-MISC" or tag == "O" else 0.01 for tag in tags}
           elif (smoothing_type==3): emission_sentence_df[word] =  {tag: 1/len(tags) for tag in tags}
           elif (smoothing_type==4): emission_sentence_df[word] =  unkown_prob
   
    return Viterbi(emission_sentence_df, transition_df)


NAIVE TAGGER

In [6]:


def naive_tagger(sentence, emission_prob):
    tags = []
    words = sentence.split()

    for word in words:
        if word in emission_prob:
            tags.append(max(emission_prob[word], key=emission_prob[word].get))
        else:
            tags.append("B-MISC")
    
    # Creazione del DataFrame
    df = pd.DataFrame({'WORD': words, 'TAG': tags})
    df['POSITION'] = df.index + 1  # Aggiunge la colonna POSITION
    df = df[['POSITION', 'WORD', 'TAG']]  # Riordina le colonne
    
    return df



Esempio di Decoding Completo

In [7]:
import memm_tagger

emission_prob=load_data("emission_prob","it")
transition_prob=load_data("transition_prob","it")
unkown_prob=load_data("unknown_prob","it")

golden_tot = read_dataset("test","it")
golden_df = golden_tot.head(100)
sentence = extract_sentences_from_dataframe(golden_df)

#carico il file di test, estraggo la sentence e pongo il test set df come golden_df
#sentence ="Il Vermont non era ancora stato colonizzato , mentre i territori del New Hampshire e del Mane erano governati dal Massachusetts ."
#golden_tag = "O,B-LOC,O,O,O,O,O,O,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,O,O,O,B-LOC,O"

#"In 1975 , the Princess was listed among women with whom actor Warren Beatty had had romantic relationships ."
#"O,O,O,O,O,O,O,O,O,O,O,O,B-PER,I-PER,O,O,O,O,O"

#golden_df = create_golden_dataframe(sentence, golden_tag)

vit_df=viterbi_tagger(sentence,emission_prob,transition_prob,unkown_prob,1)
nayve_df=naive_tagger(sentence, emission_prob)


save_to_conllu(golden_df, "golden_tag.conllu", "it")
save_to_conllu(vit_df, "viterbi_tag.conllu", "it")
save_to_conllu(nayve_df, "nayve_tag.conllu", "it")

Percorso Viterbi: [0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
DataFrame salvato in formato CoNLL-U: ../data/it/tagging/golden_tag.conllu
DataFrame salvato in formato CoNLL-U: ../data/it/tagging/viterbi_tag.conllu
DataFrame salvato in formato CoNLL-U: ../data/it/tagging/nayve_tag.conllu
